# Selenium과 BeautifulSoup을 이용한 KFC 매장 정보 수집

### 라이브러리

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import urllib.request as req
from urllib import parse
import pandas as pd
import time

### Web Driver

In [ ]:
chrome_options=webdriver.ChromeOptions()
driver=webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get('https://www.kfckorea.com/store/findStore')

___

## 매장 정보 수집

In [1]:
#지역검색: //*[@id="app"]/div[2]/div/div/div[2]/div/div[2]/div/div/ul/li[2]/a
#시도: //*[@id="app"]/div[2]/div/div/div[2]/div/div[2]/div/div/div/div/div[1]/select[1]
#구군: //*[@id="app"]/div[2]/div/div/div[2]/div/div[2]/div/div/div/div/div[1]/select[2]

region_search=driver.find_element(By.XPATH,'//*[@id="app"]/div[2]/div/div/div[2]/div/div[2]/div/div/ul/li[2]/a').click()
sido_list_raw=driver.find_element(By.XPATH,'//*[@id="app"]/div[2]/div/div/div[2]/div/div[2]/div/div/div/div/div[1]')
sido_list=sido_list_raw.find_elements(By.TAG_NAME,'option')
sido_names=[i.text for i in sido_list]
element=driver.find_element(By.XPATH, '//*[@id="app"]/div[2]/div/div/div[2]/div/div[2]/div/div/div/div/div[1]/select[1]')
time.sleep(2)
element.send_keys(sido_names[0])

address_total=[]

for l in sido_names:
    element.send_keys(l)
    gugun_names=[]
    gugun_list_raw=driver.find_element(By.XPATH,'//*[@id="app"]/div[2]/div/div/div[2]/div/div[2]/div/div/div/div/div[1]/select[2]')
    gugun_list=gugun_list_raw.find_elements(By.TAG_NAME,'option')
    gugun_names=[a.text for a in gugun_list]
    gugun_names.remove('구,군')
    print(gugun_names)
    element2=driver.find_element(By.XPATH,'//*[@id="app"]/div[2]/div/div/div[2]/div/div[2]/div/div/div/div/div[1]/select[2]')
    for i in gugun_names:
        time.sleep(1)
        element2.send_keys(i)
        html=driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        gu_address=soup.select("#app > div:nth-child(2) > div > div > div.open > div > div.detail > div > div > div > div > div:nth-child(3) > div.swiper-wrapper > div > ul > li:nth-child(3)")
        gu_address_text=[m.text for m in gu_address]
        shop_names=soup.select("#app > div:nth-child(2) > div > div > div.open > div > div.detail > div > div > div > div > div:nth-child(3) > div.swiper-wrapper > div > ul > li.top > a")
        shop_names_text=[n.text for n in shop_names]
        for j,k in zip(gu_address_text,shop_names_text):
            address_total.append((j.split()[0],j.split()[1],j,k))
            print((j.split()[0],j.split()[1],j,k))
        time.sleep(1)

[WDM] - Downloading: 100%|██████████| 6.79M/6.79M [00:00<00:00, 11.8MB/s]


[]
[]
[]


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=111.0.5563.65)
Stacktrace:
Backtrace:
	(No symbol) [0x00A7DCE3]
	(No symbol) [0x00A139D1]
	(No symbol) [0x00924DA8]
	(No symbol) [0x0090D0D3]
	(No symbol) [0x0096EA8B]
	(No symbol) [0x0097D093]
	(No symbol) [0x0096ACC6]
	(No symbol) [0x00946F68]
	(No symbol) [0x009480CD]
	GetHandleVerifier [0x00CF3832+2506274]
	GetHandleVerifier [0x00D29794+2727300]
	GetHandleVerifier [0x00D2E36C+2746716]
	GetHandleVerifier [0x00B26690+617600]
	(No symbol) [0x00A1C712]
	(No symbol) [0x00A21FF8]
	(No symbol) [0x00A220DB]
	(No symbol) [0x00A2C63B]
	BaseThreadInitThunk [0x75AE7D69+25]
	RtlInitializeExceptionChain [0x76F1B74B+107]
	RtlClearBits [0x76F1B6CF+191]


### CSV 파일로 변환

In [ ]:
df=pd.DataFrame(data=address_total,columns=['광역시도','시군구','주소','지점명'])
df.to_csv("../Data/KFC.csv",encoding='utf-8',index=False)

___
## KFC 매장의 위도와 경도 수집

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request
import json

kfc=pd.read_csv('../Data/KFC.csv')
addressesx=[]
addressesy=[]

client_secret="LfPn2DBkJs8motLtQOouNwsF6OYi7rlfzQFUWN2F"
client_id="dhnl44bahk"

for address in list(kfc['주소']):
    encText = urllib.parse.quote(address)
    url = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query=" + encText
    request = urllib.request.Request(url)
    request.add_header("X-NCP-APIGW-API-KEY-ID",client_id)
    request.add_header("X-NCP-APIGW-API-KEY",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        entext = response_body.decode('utf-8')
        data1 = json.loads(entext)
        try:
            addressesx.append(data1['addresses'][0]['x'])
            addressesy.append(data1['addresses'][0]['y'])
        except:
            addressesx.append(0)
            addressesy.append(0)
    else:
        print("Error Code:" + rescode)

print((addressesx,addressesy))

### CSV 파일로 저장

In [ ]:
kfc['경도']=addressesx
kfc['위도']=addressesy
kfc.to_csv("../Data/KFC.csv",encoding='utf-8',index=False)

### 지도에 마커로 표시

In [2]:
kfc=pd.read_csv('../Data/KFC.csv')

In [4]:
import folium

kfc_map2=folium.Map(
    location=[37.55,126.98],
    zoom_start=8,
    tiles='Stamen Terrain'
    )

for lat,lng,name in zip(kfc.위도,kfc.경도,kfc.지점명):
    folium.Marker([lat,lng],popup=name).add_to(kfc_map2)

kfc_map2